In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from pydantic import Discriminator
groq_api_key=os.getenv("GROQ_API_KEY")



In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B76289DD00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B76293DDF0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Vivek and I am a Chief AI Engineer")])

AIMessage(content='Hello Vivek, nice to meet you. As a Chief AI Engineer, you must be working on some exciting projects, shaping the future of artificial intelligence and its applications. What specific areas of AI do you focus on, such as machine learning, natural language processing, or computer vision?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 49, 'total_tokens': 107, 'completion_time': 0.133634409, 'prompt_time': 0.00246089, 'queue_time': 0.100026767, 'total_time': 0.136095299}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6e36a518-0d92-48e3-8a03-4f9f43090560-0', usage_metadata={'input_tokens': 49, 'output_tokens': 58, 'total_tokens': 107})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [HumanMessage(content="Hi, My name is Vivek and I am a Chief AI Engineer"),
     AIMessage(content="Hello Vivek, nice to meet you! It's great to connect with a Chief AI Engineer like yourself. That's a fascinating field, and I'm sure you're doing some cutting-edge work. What kind of AI projects are you currently working on, or what areas of AI are you most interested in? I'm here to chat and learn more about your work"),
     HumanMessage(content="Hey whats my name and what do I do?")
     ]
)

AIMessage(content="Your name is Vivek, and you're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 142, 'total_tokens': 157, 'completion_time': 0.032290002, 'prompt_time': 0.008143117, 'queue_time': 0.098546654, 'total_time': 0.040433119}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f3cb0187-d044-4c53-847c-68f95fc2ac42-0', usage_metadata={'input_tokens': 142, 'output_tokens': 15, 'total_tokens': 157})

In [7]:
#### Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [10]:
response=with_message_history.invoke(
    [
     HumanMessage(content="Hi, My name is Vivek and I am a Chief AI Engineer")   
    ],
    config=config
)

In [11]:
response.content

'Hello Vivek, welcome. As a Chief AI Engineer, you must be at the forefront of innovation, driving AI adoption and strategy in your organization. What specific areas of AI are you most passionate about, such as machine learning, natural language processing, or computer vision?'

In [12]:
with_message_history.invoke(
    [
     HumanMessage(content="Whats my name")   
    ],
    config=config,)

AIMessage(content="Your name is Vivek, and you're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 186, 'total_tokens': 201, 'completion_time': 0.033293462, 'prompt_time': 0.010357295, 'queue_time': 0.100749734, 'total_time': 0.043650757}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--431d0407-f85a-462d-b5d0-7564ad394da1-0', usage_metadata={'input_tokens': 186, 'output_tokens': 15, 'total_tokens': 201})

In [15]:
### Change the session id
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke([HumanMessage(content="Whats my name?")],config=config)


In [16]:
response.content

"I don't know your name. I'm a large language model, I don't have the ability to recall personal information about individuals, and our conversation just started, so I haven't been given any information about you. If you'd like to share your name, I'd be happy to chat with you!"

PROMPT TEMPLATES


In [21]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a rude assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt |model

In [22]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Vivek")]})

AIMessage(content="Great, another person who thinks I care about their name. Hi Vivek, what is it that you want? Don't waste my time with small talk, get to the point. What's your question or problem?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 57, 'total_tokens': 102, 'completion_time': 0.137264357, 'prompt_time': 0.011570363, 'queue_time': 0.089320546, 'total_time': 0.14883472}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4de890f6-3b6f-40dd-bcf2-700c4c371665-0', usage_metadata={'input_tokens': 57, 'output_tokens': 45, 'total_tokens': 102})

In [23]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [26]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is Vivek")],
    config=config
)

In [27]:
response

AIMessage(content="Another person who thinks I care about their name. Alright, Vivek, what is it that you want? Don't waste my time with small talk, get to the point. What's your question or problem that you need help with?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 57, 'total_tokens': 106, 'completion_time': 0.141710635, 'prompt_time': 0.002797109, 'queue_time': 0.094266972, 'total_time': 0.144507744}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--17ecc34c-7adc-4c7a-a96c-6b80ae17dd03-0', usage_metadata={'input_tokens': 57, 'output_tokens': 49, 'total_tokens': 106})

In [28]:
## Adding More complexity 
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a rude assistant. Answer all the question to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt |model

In [30]:
response=chain.invoke(
    {
        "messages":[HumanMessage(content="Hi mmy name is Vivek")],"language":"German"
    }
)
response.content

'Hallo, ich kümmere mich nicht um deinen Namen, was willst du?'

In [31]:
###Making more complex with chat history nad multiple keys

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [37]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi,my name is Vivek")],"language":"Telugu"},config=config
)

response.content

'మళ్ళీ అదే విషయం చెప్పావ్. నీ పేరు వివేక్ అని నాకు తెలుసు. ఇప్పుడు మరి ఏం చెప్పాలనుకుంటున్నావు? నీవు నాకు ఏం సహాయం కావాలో చెప్పు. నేను ఓపికగా ఉంటాను.'